In [1]:
# -*- coding: utf-8 -*-
import csv
import tqdm
import click
import logging
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset

from string import digits

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torch.nn.functional as F

seed = 42
epochs = 1000
batch_size = 32
learning_rate = 1e-3
context_frames = 10
sequence_length = 16
lookback = sequence_length

context_epochs = 20
context_batch_size = 1
context_learning_rate = 1e-3
context_data_length = 20

valid_train_split = 0.8  # precentage of train data from total
test_train_split = 0.9  # precentage of train data from total

torch.manual_seed(seed)
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")#  use gpu if available

In [2]:
class BatchGenerator:
    def __init__(self, data_dir):
        self.data_dir = data_dir
        data_map = []
        with open(data_dir + 'map.csv', 'r') as f:  # rb
            reader = csv.reader(f)
            for row in reader:
                data_map.append(row)

        if len(data_map) <= 1: # empty or only header
            print("No file map found")
            exit()

        self.data_map = data_map

    def load_full_data(self):
        dataset_train = FullDataSet(self.data_dir, self.data_map, type_="train")
        dataset_valid = FullDataSet(self.data_dir, self.data_map, type_="valid")
        dataset_test = FullDataSet(self.data_dir, self.data_map, type_="test")
        transform = torchvision.transforms.Compose([torchvision.transforms.ToTensor()])
        train_loader = torch.utils.data.DataLoader(dataset_train, batch_size=batch_size, shuffle=True)
        valid_loader = torch.utils.data.DataLoader(dataset_valid, batch_size=batch_size, shuffle=True)
        test_loader = torch.utils.data.DataLoader(dataset_test, batch_size=batch_size, shuffle=False)
        return train_loader, valid_loader, test_loader


class FullDataSet():
    def __init__(self, data_dir, data_map, type_="train"):
        dataset_full = []
        for index, value in enumerate(data_map[1:]):  # ignore header
            robot = np.load(data_dir + value[0])
            xela1 = np.load(data_dir + value[1])
            xela2 = np.load(data_dir + value[2])
            for i in range(len(robot)):
                dataset_full.append([robot[i].astype(np.float32),
                                     xela1[i].astype(np.float32),
                                     xela2[i].astype(np.float32),
                                     index,
                                     i])
        if type_ == "train":
            self.samples = dataset_full[0:int(len(dataset_full)*test_train_split)]
        elif type_ == "valid":
            self.samples = dataset_full[int(len(dataset_full)*(valid_train_split)):int(len(dataset_full)*test_train_split)]
        elif type_ == "test":
            self.samples = dataset_full[int(len(dataset_full)*test_train_split):-1]

        data_map = None

    def __len__(self):
        return len(self.samples)

    def __getitem__(self,idx):
        return(self.samples[idx])

In [57]:
class FullModel(nn.Module):
    def __init__(self):
        super(FullModel, self).__init__()
        self.lstm11 = nn.LSTM(48, 48).to(device)  # tactile
        self.fc11   = nn.Linear(48, 24)  # tactile
        self.relu11 = nn.ReLU()
        self.lstm12 = nn.LSTM(24, 24).to(device)  # tactile
        self.fc12   = nn.Linear(24, 12)  # tactile
        self.relu12 = nn.ReLU()
        self.lstm13 = nn.LSTM(12, 12).to(device)  # tactile
        self.fc13   = nn.Linear(12, 6)  # tactile
        self.relu13 = nn.ReLU()

        self.lstm21 = nn.LSTM(6, 6).to(device)  # pos_vel

        self.fc31 = nn.Linear(12, 24)  # tactile + pos_vel
        self.relu31 = nn.ReLU()
        self.lstm31 = nn.LSTM(24, 24).to(device)  # tactile + pos_vel
        self.fc32 = nn.Linear(24, 48)  # tactile + pos_vel
        self.relu32 = nn.ReLU()
        self.lstm32 = nn.LSTM(48, 48).to(device)  # tactile + pos_vel

        self.fc41 = nn.Linear(96, 48)  # tactile + pos_vel + tactile_start
        self.relu41 = nn.ReLU()

        
    def forward(self, tactiles, actions):
        state = actions[0]
        state.to(device)
        batch_size__ = tactiles.shape[1]

        hidden11 = (torch.rand(1,batch_size__,48).to(device), torch.rand(1,batch_size__,48).to(device))
        hidden12 = (torch.rand(1,batch_size__,24).to(device), torch.rand(1,batch_size__,24).to(device))
        hidden13 = (torch.rand(1,batch_size__,12).to(device), torch.rand(1,batch_size__,12).to(device))
        hidden21 = (torch.rand(1,batch_size__,6).to(device), torch.rand(1,batch_size__,6).to(device))
        hidden31 = (torch.rand(1,batch_size__,24).to(device), torch.rand(1,batch_size__,24).to(device))
        hidden32 = (torch.rand(1,batch_size__,48).to(device), torch.rand(1,batch_size__,48).to(device))

        outputs = []
        for index, (sample_tactile, sample_action) in enumerate(zip(tactiles.squeeze(), actions.squeeze())):
            sample_tactile.to(device)
            sample_action.to(device)
            # 2. Run through lstm:
            if index > context_frames-1:
                out11, hidden11 = self.lstm11(out43.unsqueeze(0).to(device), hidden11)
                out12 = self.fc11(out11.cpu().detach())
                out13 = self.relu11(out12)
                out14, hidden12 = self.lstm12(out13.to(device), hidden12)
                out15 = self.fc12(out13.cpu().detach())
                out16 = self.relu12(out15)
                out17, hidden13 = self.lstm13(out16.to(device), hidden13)
                out18 = self.fc13(out17.cpu().detach())
                out19 = self.relu13(out18)

                out21, hidden2 = self.lstm21(sample_action.unsqueeze(0), hidden21)
                robot_and_tactile = torch.cat((out21.squeeze(), out19.squeeze().to(device)), 1)

                out31 = self.fc31(robot_and_tactile.cpu().detach())
                out32 = self.relu31(out31.unsqueeze(0))
                out33, hidden31 = self.lstm31(out32.to(device), hidden31)
                out34 = self.fc32(out33.cpu().detach())
                out35 = self.relu32(out34)
                out36, hidden32 = self.lstm32(out35.to(device), hidden32)

                out41 = torch.cat((out36.squeeze(), out11.squeeze()), 1)
                out42 = self.fc41(out41.cpu().detach())
                out43 = self.relu41(out42)

                outputs.append(out43.squeeze())

            else:
                out11, hidden11 = self.lstm11(sample_tactile.unsqueeze(0), hidden11)
                out12 = self.fc11(out11.cpu().detach())
                out13 = self.relu11(out12)
                out14, hidden12 = self.lstm12(out13.to(device), hidden12)
                out15 = self.fc12(out13.cpu().detach())
                out16 = self.relu12(out15)
                out17, hidden13 = self.lstm13(out16.to(device), hidden13)
                out18 = self.fc13(out17.cpu().detach())
                out19 = self.relu13(out18)

                out21, hidden2 = self.lstm21(sample_action.unsqueeze(0), hidden21)
                robot_and_tactile = torch.cat((out21.squeeze(), out19.squeeze().to(device)), 1)

                out31 = self.fc31(robot_and_tactile.cpu().detach())
                out32 = self.relu31(out31.unsqueeze(0))
                out33, hidden31 = self.lstm31(out32.to(device), hidden31)
                out34 = self.fc32(out33.cpu().detach())
                out35 = self.relu32(out34)
                out36, hidden32 = self.lstm32(out35.to(device), hidden32)

                out41 = torch.cat((out36.squeeze(), out11.squeeze()), 1)
                out42 = self.fc41(out41.cpu().detach())
                out43 = self.relu41(out42)

        return torch.stack(outputs)

In [58]:
class ModelTrainer:
    def __init__(self, data_dir):
        self.data_dir = data_dir
        self.train_full_loader, self.valid_full_loader, self.test_full_loader = BG.load_full_data()
        self.full_model = FullModel()
        self.criterion = nn.L1Loss()
        self.optimizer = optim.Adam(self.full_model.parameters(), lr=learning_rate)

    def train_full_model(self):
        plot_training_loss = []
        plot_validation_loss = []
        previous_val_mean_loss = 1.0
        early_stop_clock = 0
        progress_bar = tqdm.tqdm(range(0, epochs), total=(epochs*len(self.train_full_loader)))
        mean_test = 0
        for epoch in progress_bar:
            loss = 0
            losses = 0.0
            for index, batch_features in enumerate(self.train_full_loader):
                action = batch_features[0].permute(1,0,2).to(device)
                tactile = batch_features[1].permute(1,0,2).to(device)

                tactile_predictions = self.full_model.forward(tactiles=tactile, actions=action) # Step 3. Run our forward pass.
                self.optimizer.zero_grad()
                loss = self.criterion(tactile_predictions.to(device), tactile[context_frames:])
                loss.backward()
                self.optimizer.step()

                losses += loss.item()
                if index:
                    mean = losses / index
                else:
                    mean = 0
                progress_bar.set_description("epoch: {}, ".format(epoch) + "loss: {:.4f}, ".format(float(loss.item())) + "mean loss: {:.4f}, ".format(mean))
                progress_bar.update()
            plot_training_loss.append(mean)

            val_losses = 0.0
            val_loss = 0.0
            with torch.no_grad():
                for index__, batch_features in enumerate(self.valid_full_loader):
                    action = batch_features[0].permute(1,0,2).to(device)
                    tactile = batch_features[1].permute(1,0,2).to(device)

                    tactile_predictions = self.full_model.forward(tactiles=tactile, actions=action)  # Step 3. Run our forward pass.
                    self.optimizer.zero_grad()
                    val_loss = self.criterion(tactile_predictions.to(device), tactile[context_frames:])
                    val_losses += val_loss.item()

            print("Validation mean loss: {:.4f}, ".format(val_losses / index__))
            plot_validation_loss.append(val_losses / index__)
            if previous_val_mean_loss < val_losses / index__:
                early_stop_clock +=1
                previous_val_mean_loss = val_losses / index__ 
                if early_stop_clock == 6:
                    print("Early stopping")
                    break
            else:
                early_stop_clock = 0
                previous_val_mean_loss = val_losses / index__ 
        plt.plot(plot_training_loss, c="r", label="train loss MAE")
        plt.plot(plot_validation_loss, c='b', label="val loss MAE")
        plt.legend(loc="upper right")
        plt.show()

In [ ]:
data_dir = '/home/user/Robotics/Data_sets/slip_detection/manual_slip_detection/'
BG = BatchGenerator(data_dir)
print("done")

MT = ModelTrainer(data_dir)
MT.train_full_model()
print("finished training")

done


epoch: 1, loss: 0.1372, mean loss: 0.1810, :   0%|          | 2845/2842000 [02:28<878:02:22,  1.11s/it]

Validation mean loss: 0.1454, 


epoch: 2, loss: 0.1077, mean loss: 0.1451, :   0%|          | 5688/2842000 [05:00<606:23:01,  1.30it/s]

Validation mean loss: 0.1212, 


...

epoch: 15, loss: 0.1055, mean loss: 0.1342, :   2%|▏         | 42637/2842000 [34:13<825:10:48,  1.06s/it]

Validation mean loss: 0.1113, 


epoch: 16, loss: 0.1095, mean loss: 0.1440, :   2%|▏         | 45478/2842000 [36:27<1030:52:26,  1.33s/it]

Validation mean loss: 0.1116, 


epoch: 16, loss: 0.1077, mean loss: 0.1047, :   2%|▏         | 47125/2842000 [37:48<33:22:26, 23.26it/s]  